# HW5 - Text classification with an RNN
### 0858610 周承翰
參考連結:https://www.tensorflow.org/tutorials/text/text_classification_rnn

### Setup

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

AttributeError: module 'tensorflow._api.v2.autograph.experimental' has no attribute 'do_not_convert'

In [2]:
tf.__version__

NameError: name 'tf' is not defined

In [3]:
tf.test.is_gpu_available()

True

### import matplotlib and create a helper function to plot graphs:

In [4]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string], '')
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

### Setup input pipeline
The IMDB large movie review dataset is a binary classification dataset—all the reviews have either a positive or negative sentiment.<br />

Download the dataset using TFDS.<br />

使用tfds.load load 資料imdb


In [5]:
dataset, info = tfds.load('imdb_reviews/subwords8k',
                          shuffle_files=True,
                          with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

The dataset info includes the encoder (a tfds.features.text.SubwordTextEncoder).

In [6]:
encoder = info.features['text'].encoder

In [7]:
print ('Vocabulary size: {}'.format(encoder.vocab_size))


Vocabulary size: 8185


This text encoder will reversibly encode any string, falling back to byte-encoding if necessary.


In [8]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [9]:
# 檢查跟原始字串有沒有一樣
assert original_string == sample_string

In [10]:
for index in encoded_string:
  print ('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


### Prepare the data for training
Next create batches of these encoded strings. Use the padded_batch method to zero-pad the sequences to the length of the longest string n the batch:<br />

Dataset.shuffle() 與 tf.RandomShuffleQueue使用相同的算法對資料夾隨機shuffle：它有一個固定大小的buffer，并從該buffer中随机均匀地選擇下一個元素 <br />
Dataset.padded_batch() 允許你將不同shape的tensors進行batch，通过指定一或多个dimensions，在其上进行pad。

In [11]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [12]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)


### Create the model

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

### Training Model

In [18]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

Epoch 1/10
      1/Unknown - 1s 1s/step

CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node Reshape_11/_38}}]] [Op:__inference_distributed_function_6337]

Function call stack:
distributed_function
